# ABC Analysis

In [107]:
import pandas as pd

In [108]:
df = pd.read_csv(f'ABC Analysis\INPUT_sales.csv', usecols=['StockCode','Quantity','Price']) 
df.head(2)

,StockCode,Quantity,Price
0,85048,12,6.95
1,79323P,12,6.75


In [109]:
#kiểm tra null & type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044420 entries, 0 to 1044419
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   StockCode  1044420 non-null  object 
 1   Quantity   1044420 non-null  int64  
 2   Price      1044420 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 23.9+ MB


In [110]:
df.isna().sum()
#không có missing values

StockCode    0
Quantity     0
Price        0
dtype: int64

In [111]:
df[df['Price']<0]
#nhận thấy có 5 StockCode có Price < 0 -> drop

,StockCode,Quantity,Price
174809,B,1,-53594.36
268853,B,1,-44031.79
393373,B,1,-38925.87
806755,B,1,-11062.06
806756,B,1,-11062.06


In [112]:
df[df['Quantity']<0]
#không có vấn đề

,StockCode,Quantity,Price


In [113]:
df['StockCode'].value_counts() 

85123A    5692
85099B    4129
22423     4065
21212     3290
20725     3179
          ... 
21406        1
35818C       1
35950        1
84205C       1
23843        1
Name: StockCode, Length: 4985, dtype: int64

In [123]:
df_clean = df[df['Price']>=0]
df_clean.head(2)

,StockCode,Quantity,Price
0,85048,12,6.95
1,79323P,12,6.75


In [124]:
df_clean['Revenue'] = df_clean['Price'] * df_clean['Quantity']

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7256\3362176084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Revenue'] = df_clean['Price'] * df_clean['Quantity']


In [125]:
df_clean = df_clean.groupby('StockCode')['Revenue'].sum().reset_index()

In [126]:
df_clean = df_clean.sort_values('Revenue', ascending = False)

In [127]:
total_revenue = df_clean['Revenue'].sum()

In [128]:
df_clean['% Cumulative Revenue'] = (df_clean['Revenue'].cumsum())/total_revenue

In [129]:
df_clean['Revenue_ratio'] = df_clean['Revenue']/total_revenue
df_clean.head(2)

,StockCode,Revenue,% Cumulative Revenue,Revenue_ratio
1655,22423,344563.25,0.016429,0.016429
4968,M,340716.28,0.032675,0.016246


In [156]:
df_clean['Class'] = pd.cut(x = df_clean['% Cumulative Revenue'], bins = [0, 0.8, 0.95, 1.000001],
       labels = ['A', 'B', 'C'])
df_clean.head(5)
# lựa chọn bins cuối chặn 1.000001 do SKUs gần cuối của class C quá bé 
#-> %Cum đều bằng 1% -> để bins chặn 1 thì Python không tính

,StockCode,Revenue,% Cumulative Revenue,Revenue_ratio,Class
1655,22423,344563.25,0.016429,0.016429,A
4968,M,340716.28,0.032675,0.016246,A
4967,DOT,322657.48,0.048060,0.015385,A
4364,85123A,263109.67,0.060605,0.012545,A
4336,85099B,183454.83,0.069352,0.008747,A


In [157]:
df_final = df_clean.groupby('Class')[['StockCode', 'Revenue_ratio']].agg(
    {'StockCode':'count','Revenue_ratio':'sum'})

In [158]:
total_SKUs = df_clean['StockCode'].count()
total_SKUs

4985

In [159]:
df_final['SKU_ratio'] = df_final['StockCode']/total_SKUs
df_final = df_final[['StockCode','SKU_ratio','Revenue_ratio']]
df_final

,StockCode,SKU_ratio,Revenue_ratio
Class,,,
A,1004,0.201404,0.799970
B,1285,0.257773,0.149996
C,2696,0.540822,0.050034


In [160]:
df_final.rename(columns = {'StockCode':'Number of SKUs'}, inplace = True)
df_final

,Number of SKUs,SKU_ratio,Revenue_ratio
Class,,,
A,1004,0.201404,0.799970
B,1285,0.257773,0.149996
C,2696,0.540822,0.050034
